In [ ]:
import pandas as pd
book=pd.read_csv('../input/toy-dataset/toy_dataset.csv')
book.head()

In [ ]:
book.nunique()

In [ ]:
book.info()

In [ ]:
book.skew()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs = axs.flatten()
sns.boxplot(y=book['Income'],ax=axs[0])
sns.boxplot(y=book['Age'],ax=axs[1])
plt.show()

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(book, title="EDA Report")

In [ ]:
# book=book.sample(frac =.05)
# book.head()

In [ ]:
round(book.Illness.value_counts()*100/len(book),2)

In [ ]:
round(book.City.value_counts()*100/len(book),2)

In [ ]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy1 = pd.get_dummies(book[['Gender','City']], drop_first=True)
dummy1.head()

In [ ]:
# Adding the results to the master dataframe
book = pd.concat([book, dummy1], axis=1)
book.head()

In [ ]:
book = book.drop(['Gender','City','Number'], axis = 1)
book.head()

In [ ]:
book=book.replace({'Illness' : { 'No' : 0, 'Yes' : 1 }})
book.head()

In [ ]:
book.skew()

In [ ]:
!pip install --upgrade git+https://github.com/tusharsarkar3/XBNet.git

This is based on paper named "XBNet : An Extremely Boosted Neural Network"

Link : https://arxiv.org/abs/2106.05239

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET 

In [ ]:
y=book[['Illness']]
y.head()

In [ ]:
x=book.drop(['Illness'],axis=1)
x.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.2, random_state= True,stratify=y) 
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
x_train_smote, y_train_smote = sm.fit_resample(x_train, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_smote=scaler.fit_transform(x_train_smote)
x_test=scaler.transform(x_test)

In [ ]:
y_train_smote=y_train_smote.to_numpy()
y_train_smote[:5]

In [ ]:
y_test=y_test.to_numpy()
y_test[:5]

In [ ]:
y_train_smote=y_train_smote.reshape((-1))
y_train_smote.shape

In [ ]:
y_test=y_test.reshape((-1))
y_test.shape

In [ ]:
model = XBNETClassifier(x_train_smote,y_train_smote,num_layers=2)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02) 

In [ ]:
m,acc, lo, val_ac, val_lo = run_XBNET(x_train_smote,x_test,y_train_smote,y_test,model,criterion,
                                      optimizer,epochs=24,batch_size=10*len(book))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(acc,label='training accuracy')
plt.plot(val_ac,label = 'validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.grid()
plt.subplot(1,2,2)
plt.plot(lo,label='training loss')
plt.plot(val_lo,label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend() 
plt.grid()